# ☁️ AWS para Ingeniería de Datos: S3, Glue, Athena y Lambda

Este notebook introduce un flujo moderno de datos en AWS con almacenamiento en S3, transformación con Glue (PySpark), consulta con Athena (SQL sobre S3) y orquestación con eventos/Lambda. Incluye ejemplos de código con `boto3` y prácticas recomendadas.

## Requisitos y Notas de Ejecución

- Para ejecutar código real de AWS necesitas credenciales configuradas en tu entorno (`AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, `AWS_REGION`).
- Nunca subas credenciales al repositorio. Usa variables de entorno o perfiles de AWS CLI.
- Este notebook es auto-contenido con bloques que pueden ejecutarse si tu entorno ya tiene permisos.
- Alternativa local: usar LocalStack para simular servicios AWS en tu máquina.

## 1. S3: Data Lake Básico

In [ ]:
import os, json, io
import pandas as pd
# import boto3  # Descomenta si tienes credenciales configuradas

BUCKET = 'mi-data-lake-demo-123456'  # Cambia por un nombre único global
PREFIX_RAW = 'raw/ventas/'
PREFIX_CURATED = 'curated/ventas/'

print('👆 Define el bucket y prefijos antes de ejecutar contra AWS')

### 1.1 Crear bucket (opcional)

In [ ]:
# s3 = boto3.client('s3')
# try:
#     s3.create_bucket(Bucket=BUCKET, CreateBucketConfiguration={'LocationConstraint': 'us-east-1'})
#     print('✅ Bucket creado')
# except s3.exceptions.BucketAlreadyOwnedByYou:
#     print('ℹ️ Bucket ya existe')
# except Exception as e:
#     print('❌ Error creando bucket:', e)

### 1.2 Subir dataset de ejemplo a S3

In [ ]:
# df = pd.read_csv('../../datasets/raw/ventas.csv')
# csv_bytes = df.to_csv(index=False).encode('utf-8')
# s3.put_object(Bucket=BUCKET, Key=PREFIX_RAW + 'ventas_2025_10.csv', Body=csv_bytes)
# print('📤 Archivo subido a S3')

## 2. Glue: Transformaciones con PySpark (Job Script)

Ejemplo de script de Glue (PySpark) para leer CSVs crudos de S3, limpiar/transformar y escribir en formato Parquet particionado por mes. Guarda este script como `glue_job.py` y súbelo a un Job de Glue.

In [ ]:
glue_job_script = r'''
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql import functions as F

args = getResolvedOptions(sys.argv, ['JOB_NAME','BUCKET','PREFIX_RAW','PREFIX_CURATED'])
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

raw_path = f